## **Homework Exercise**

### **Goal:** to invert synthetic S-P travel time differences to determine the optimal position of epicenter and optimal parameter of the wave propagation: the slowness difference between *S* and *P* waves **Sdiff**.

#### The synthetic data may contain outliers that should be removed during the analysis
<br>

### *There are 3 different datasets to analyze*

---



## Main definitions
---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as col
from scipy.interpolate import interp1d

#------------------ plotting mode
%matplotlib widget
#----------------------------

plt.close("all")

#---------------------------------------
# function to compute misfit (cost function)
#---------------------------------------
def cost_f(x0,y0,xsta,ysta, s_p_obs, sdiff, sigma):
    dist = np.sqrt((xsta-x0)**2 + (ysta-y0)**2)
    s_p = dist*sdiff
    ns = s_p_obs.size
    cf1 = np.sum(((s_p-s_p_obs)**2))/(ns*sigma**2)
    cf2 = np.exp(-1/2.*np.sum(((s_p-s_p_obs)**2))/(ns*sigma**2))
    cf3 = np.sum(np.exp(-1/2.*((s_p-s_p_obs)**2)/(sigma**2)))/ns
    return (cf1,cf2,cf3)
#---------------------------------------


#---------------------------------------
# defining parameters
#---------------------------------------
xpmin = -10         # limits for the grid search
xpmax = 10
ypmin = -10
ypmax = 10


dx = .1             # steps for the grid search
dy = .1

#------------
print('functions defined')



---
## **DATASET 1**
---

In [ ]:
#---------------------------------------
# station positions and trave times
#---------------------------------------
xsta = np.array([-0.86143713, -6.75879026, -4.63915383,  0.56571361, -8.61519761, -7.45851024,  9.05889477, -4.54968806,  4.97434889,  9.03859529])
ysta = np.array([ 3.33032529,  4.56246923, -6.40607002, -5.9906447 , -4.40832454, -1.37153873, -1.77089045,  9.77890676,  2.07250531,  4.36442152])
s_p_obs = np.array([41.50138361, 94.94711364, 102.71159377, 52.92566682, 117.20488086, 99.04538568,   75.32022778,  111.46761586, 30.20794795, 74.64174122])


#------------
print('DATASET 1 selected')



---
## **DATASET 2**
---

In [ ]:
#---------------------------------------
# station positions and trave times
#---------------------------------------
xsta = np.array([-0.57085642, -3.28930272,  0.59536097, -5.1166107 , -0.71900132, 1.00922644,  9.41697406,  9.35757605, -1.06934223,  5.02907055])
ysta = np.array([-0.95283981, -5.56197313, -7.80887777, -6.94720526, -5.71517172, -2.03448918,  1.26432628,  6.54229107,  8.8449601 ,  2.40334336])
s_p_obs = np.array([  30.52121241,   81.05167391,  111.40027683,  105.70413953, 83.49426763,   51.7971937 ,  134.94028608,  148.47611497, 100.22286391,   88.34601555])

#------------
print('DATASET 2 selected')



---
## **DATASET 3**
---

In [ ]:
#---------------------------------------
# station positions and trave times
#---------------------------------------
xsta = np.array([ 6.33423378,  0.68174178, -9.90257011,  8.04665788, -6.35928144, 6.606948  ,  4.91262369,  6.80722781, -2.59808543, -0.0141221 ])
ysta = np.array([-1.01406604, -5.45958012, -8.75728634,  8.10994188,  9.01552554, -1.67393877, -0.54573501, -8.60724762, -7.17790188,  9.31358189])
s_p_obs = np.array([  64.45071502,   38.8281537 ,  102.36075738,  108.67181587, 108.5121056 ,   67.54135551,   56.00512405,   91.60663983, 51.27181389,   84.98534941])

#------------
print('DATASET 3 selected')



---
## Inversion and plotting
---

In [ ]:
#---------------------------------------
# Selecting the value of Sdiff
#---------------------------------------

sdiff = 10.

sigmat = 4.



nx = int((xpmax-xpmin)/dx) + 1      # defining grid for search
ny = int((ypmax-ypmin)/dy) + 1

xpmax = xpmin + (nx-1)*dx
ypmax = ypmin + (ny-1)*dy

mf1 = np.zeros((nx, ny))
mf2 = np.zeros((nx, ny))
mf3 = np.zeros((nx, ny))
xm = np.zeros((nx, ny))
ym = np.zeros((nx, ny))


for ix in range(0, nx):                     # grid search
    for iy in range(0, ny):
        xm[ix,iy] = xpmin + ix*dx
        ym[ix,iy] = ypmin + iy*dy
        (mf1[ix,iy],mf2[ix,iy],mf3[ix,iy]) = cost_f(xm[ix,iy],ym[ix,iy],xsta,ysta, s_p_obs, sdiff, sigmat)
        
        
mf = mf2                                    # selecting final type of the cost function


epicenter = np.where(mf == mf.max())        # finding "best-fit" epicenter
xE = xm[epicenter]
yE = ym[epicenter]
mfE = mf[epicenter]



plt.close('all')

#---------------------------------------
# plotting results
#---------------------------------------
fig1 = plt.figure(1, figsize=(8,8))
ax = fig1.add_axes([0.1,0.1,0.8,0.8])

my_cmap = plt.cm.gist_stern_r
cs = plt.pcolor(xm, ym, mf, cmap=my_cmap)
cbar = plt.colorbar(orientation='horizontal', pad=0.05)
cbar.set_label('cost function')

plt.plot(xsta,ysta,'.')
plt.plot(xE,yE,'y*')


#-----------------------------------------------
# plotting residuals at the best fit position
#-----------------------------------------------
d = np.sqrt((xsta-xE)**2 + (ysta-yE)**2)
t_s_p = d*sdiff

resid = t_s_p - s_p_obs
mean_resid = np.mean(resid)
std_resid = np.std(resid)

midline = resid-resid + mean_resid
maxline1 = midline + std_resid
maxline2 = midline + 2*std_resid
minline1 = midline - std_resid
minline2 = midline - 2*std_resid

plt.show()


#---------------------------------------
fig2 = plt.figure(2, figsize=(8,8))
ax = fig2.add_axes([0.1,0.1,0.8,0.6])

plt.plot(resid,'ok', mfc='none')
plt.plot(midline,'g')
plt.plot(maxline1,'g--')
plt.plot(maxline2,'r--')
plt.plot(minline1,'g--')
plt.plot(minline2,'r--')
plt.xlim(-1,np.size(resid))
plt.xlabel('station N')
plt.ylabel('residual (s)')
plt.title('Summary of misfit and residuals at the best-fit location\n\
best-fit X %.2f\n\
best-fit Y %.2f\n\
cost function %.7f\n\
mean time residual (s) %.2f\n\
std of the time residuals (s) %.2f' % (xE, yE, mfE, mean_resid, std_resid) )

plt.show()



#-----------------------------------------------
# analyzing residuals for outliers
#-----------------------------------------------
nr = np.size(resid)

out = np.zeros(nr)

num = np.arange(nr)

for i in range(0, nr):
    numi = num != i
    m = np.mean(resid[numi])
    s = np.std(resid[numi])
    out[i] = np.fabs(resid[i]-m)/s

maxline = resid-resid + 3.
ymax = np.max((np.max(out),3)) +.25


#-----------------------------------------------
fig3 = plt.figure(3, figsize=(8,8))
ax = fig3.add_axes([0.1,0.1,0.8,0.8])

plt.plot(out,'ok', mfc='none')
plt.plot(maxline,'r--')
plt.xlim(-1,nr)
plt.ylim(0,ymax)
plt.xlabel('station N')
plt.ylabel('residual in standard deviations')

plt.show()



#-----------------------------------------------
fig4 = plt.figure(4)
plt.hist(resid)
plt.ylabel('histogram')
plt.xlabel('residual(s)')

plt.show()

